# Sleep Efficiency Prediction using a Feed-Forward Neural Network
This notebook demonstrates a modularized approach to predicting sleep efficiency based on multiple factors. The workflow includes loading data, preprocessing it, defining a neural network model, training, and evaluating the model.

In [ ]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from google.colab import files


## Step 1: Upload and Load the Dataset
Upload the `Sleep_Efficiency.csv` file to start processing the data.

In [ ]:

uploaded = files.upload()


file_path = 'Sleep_Efficiency.csv'
data = pd.read_csv(file_path)
data.head()

Saving Sleep_Efficiency.csv to Sleep_Efficiency.csv


,ID,Age,Gender,Bedtime,Wakeup time,Sleep duration,Sleep efficiency,REM sleep percentage,Deep sleep percentage,Light sleep percentage,Awakenings,Caffeine consumption,Alcohol consumption,Smoking status,Exercise frequency
0,1,65,Female,2021-03-06 01:00:00,2021-03-06 07:00:00,6.0,0.88,18,70,12,0.0,0.0,0.0,Yes,3.0
1,2,69,Male,2021-12-05 02:00:00,2021-12-05 09:00:00,7.0,0.66,19,28,53,3.0,0.0,3.0,Yes,3.0
2,3,40,Female,2021-05-25 21:30:00,2021-05-25 05:30:00,8.0,0.89,20,70,10,1.0,0.0,0.0,No,3.0
3,4,40,Female,2021-11-03 02:30:00,2021-11-03 08:30:00,6.0,0.51,23,25,52,3.0,50.0,5.0,Yes,1.0
4,5,57,Male,2021-03-13 01:00:00,2021-03-13 09:00:00,8.0,0.76,27,55,18,3.0,0.0,3.0,No,3.0


## Step 2: Data Preprocessing
In this step, we clean the data, encode categorical features, handle missing values, and split the data into training and testing sets.

In [ ]:

def load_and_preprocess_data(file_path):
    data = pd.read_csv(file_path)
    data_cleaned = data.drop(columns=["ID", "Bedtime", "Wakeup time"]).dropna()


    label_encoder = LabelEncoder()
    data_cleaned['Gender'] = label_encoder.fit_transform(data_cleaned['Gender'])
    data_cleaned['Smoking status'] = label_encoder.fit_transform(data_cleaned['Smoking status'])


    X = data_cleaned.drop(columns=["Sleep efficiency"]).values
    y = data_cleaned["Sleep efficiency"].values


    print("Target values range:", y.min(), "to", y.max())


    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    return X_train_scaled, X_test_scaled, y_train, y_test


X_train_scaled, X_test_scaled, y_train, y_test = load_and_preprocess_data(file_path)

Target values range: 0.5 to 0.99


## Step 3: Define the Neural Network Model
We define a simple feed-forward neural network model to predict sleep efficiency.

In [ ]:

class NeuralNetworkModel:
    def __init__(self, input_dim, learning_rate=0.001):
        self.model = Sequential([
            Dense(64, input_dim=input_dim, activation='relu'),
            Dense(32, activation='relu'),
            Dense(1)
        ])
        self.model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=learning_rate))

    def train(self, X_train, y_train, X_val, y_val, epochs=20, batch_size=10):
        return self.model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val))

    def evaluate(self, X_test, y_test):
        test_loss = self.model.evaluate(X_test, y_test)
        print(f"Test Loss (Mean Squared Error): {test_loss}")
        return test_loss


## Step 4: Train the Model
Initialize the model and train it on the processed data.

In [ ]:

model = NeuralNetworkModel(input_dim=X_train_scaled.shape[1], learning_rate=0.001)
history = model.train(X_train_scaled, y_train, X_test_scaled, y_test, epochs=20, batch_size=10)

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.2460 - val_loss: 0.0584
Epoch 2/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0351 - val_loss: 0.0282
Epoch 3/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0195 - val_loss: 0.0209
Epoch 4/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0145 - val_loss: 0.0174
Epoch 5/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0143 - val_loss: 0.0152
Epoch 6/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0104 - val_loss: 0.0136
Epoch 7/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0083 - val_loss: 0.0136
Epoch 8/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0072 - val_loss: 0.0124
Epoch 9/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0060 - val_loss: 0.0122
Epoch 10/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0051 - val_loss: 0.0116
Epoch 11/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0050 - val_loss: 0.0120
Epoch 12/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0038 - val_loss: 0.0108


## Step 5: Evaluate the Model
Check the model's performance on the test data.

In [ ]:

model.evaluate(X_test_scaled, y_test)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0104 
Test Loss (Mean Squared Error): 0.010224184021353722


0.010224184021353722